In [ ]:
import numpy as np

# Algoritmi

## Funzioni

### Zeri di funzione

In [10]:
def metodo_bisezione(fname, a, b, tolx): 
    fa=fname(a)
    fb=fname(b)
    if np.sign(fa)*np.sign(fb) > 0:
        print("Non è possibile applicare il metodo di bisezione \n")
        return None, None,None
    
    it = 0
    v_xk = []
    erroreX = tolx+1
    maxit = np.ceil(np.log2(np.abs(b-a)/tolx)-1)
    
    while it <= maxit and erroreX >= tolx:
        xk = a + (b-a)/2 # più stabile rispetto (b+a)/2
        v_xk.append(xk)
        it += 1
        fxk=fname(xk)
        if fxk==0:
            return xk, it, v_xk
        
        if np.sign(fb)*np.sign(fxk) <= 0: # radice nel range [xk, b]
            a = xk
            fa= fxk 
        elif np.sign(fa)*np.sign(fxk) <= 0: # radice nel range [a, xk]
            b = xk
            fb= fxk

        erroreX = np.abs(b-a)

    return xk, it, v_xk